In [4]:
# In this case the local root of the repo is our working directory
DIRECTORY = './'
font = 'arial'


from Library.Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(V, Stats, model): 
    # printing Stats
    print("R2 = %.2f (+/- %.2f) Constraint = %.2f (+/- %.2f)" % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    Vout = tf.convert_to_tensor(np.float32(model.Y))
    Loss_norm, dLoss = Loss_Vout(V, model.Pout, Vout)
    print('Loss Targets', np.mean(Loss_norm))
    Loss_norm, dLoss = Loss_SV(V, model.S)
    print('Loss SV', np.mean(Loss_norm))
    Vin = tf.convert_to_tensor(np.float32(model.X))
    Pin = tf.convert_to_tensor(np.float32(model.Pin))
    if Vin.shape[1] == model.S.shape[1]: # special case
        Vin  = tf.linalg.matmul(Vin, tf.transpose(Pin), b_is_sparse=True)
    Loss_norm, dLoss = Loss_Vin(V, model.Pin, Vin, model.mediumbound,model)
    print('Loss Vin bound', np.mean(Loss_norm))
    Loss_norm, dLoss = Loss_Vpos(V, model)
    print('Loss V positive', np.mean(Loss_norm))

# Mechanistic Models

# Examples with non-trainable mechanistic models, using FBA simulated training sets

In both LP and QP solver, one can change the `trainname` suffix (EB or UB) to use exact or upper bounds as inputs.

## LP solver

In [6]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using E. coli core simulation training sets and EB (or UB) bounds

# What you can change
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_EB' # the training set file name
size = 10 # number of runs must be lower than the number of element in trainname
timestep = int(1.0e4) # LP 1.0e4 QP 1.0e5
learn_rate = 0.3 # LP 0.3 QP 1.0
decay_rate = 0.9 # only in QP, UB 0.333 EB 0.9
# End of What you can change

# Create model and run GD for X and Y randomly drawn from trainingfile
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ecoli_core_w_GAM'], 
              model_type = 'MM_LP', 
              timestep = timestep, 
              learn_rate = learn_rate, 
              decay_rate = decay_rate)

# Select a random subset of the training set (of specified size)
# With LP we also have to change b_ext and b_int accordingly
ID = np.random.choice(model.X.shape[0], size, replace=False)
model.X, model.Y = model.X[ID,:], model.Y[ID,:]
if model.mediumbound == 'UB':
    model.b_ext = model.b_ext[ID,:]
if model.mediumbound == 'EB':
    model.b_int = model.b_int[ID,:]

# Prints a summary of the model before running
model.printout()

# Runs the appropriate method
Ypred, Stats = MM_LP(model, verbose=True)

# Printing results
printout(Ypred, Stats, model)

training file: ./Dataset_model/e_coli_core_EB
model type: MM_LP
model scaler: 0.0
model input dim: 20
model output dim: 1
model medium bound: EB
timestep: 10000
training set size (10, 20) (10, 1)
LP-Loss 1 0.2272104 0.07015557
LP-Loss 10 0.15282075 0.055338588
LP-Loss 100 0.05627328 0.031071583
LP-Loss 1000 0.00066465396 0.00017357188
LP-Loss 2000 6.618473e-05 0.000107279615
LP-Loss 3000 1.7334908e-06 3.7169455e-06
LP-Loss 4000 4.3478298e-07 1.041795e-06
LP-Loss 5000 1.3090464e-07 2.7213605e-07
LP-Loss 6000 9.06163e-08 2.6308328e-07
LP-Loss 7000 8.9164594e-08 2.672046e-07
LP-Loss 8000 2.6347433e-08 7.9037285e-08
LP-Loss 9000 3.8238204e-10 1.1460827e-09
LP-Loss 10000 2.013187e-12 5.1632383e-12
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (10, 1) (10, 1) (10, 1) (10, 1) (10, 154) (10, 312)
R2 = 1.00 (+/- 0.00) Constraint = 0.00 (+/- 0.00)
Loss Targets 1.5377998e-06
Loss SV 6.4773485e-07
Loss Vin bound 0.0
Loss V positive 7.3044006e-09


In [14]:
Ypred.shape, Stats.test_loss, Stats.test_objective

((10, 154), (0, 0), (0, 0))

## QP solver

In [16]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using E. coli core simulation training sets and EB (or UB) bounds

# What you can change
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_EB' # the training set file name
size = 10 # number of runs must be lower than the number of element in trainname
timestep = int(1.0e5) # LP 1.0e4 QP 1.0e5
learn_rate = 1.0 # LP 0.3 QP 1.0
decay_rate = 0.9 # only in QP, UB 0.333 EB 0.9
# End of What you can change

# Create model and run GD for X and Y randomly drawn from trainingfile
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ecoli_core_w_GAM'], 
              model_type = 'MM_QP', 
              timestep = timestep, 
              learn_rate = learn_rate, 
              decay_rate = decay_rate)

# Select a random subset of the training set (of specified size)
ID = np.random.choice(model.X.shape[0], size, replace=False)
model.X, model.Y = model.X[ID,:], model.Y[ID,:]

# Prints a summary of the model before running
model.printout()

# Runs the appropriate method
if model.model_type is 'MM_QP':
    Ypred, Stats = MM_QP(model, verbose=True)

# Printing results
printout(Ypred, Stats, model)

training file: ./Dataset_model/e_coli_core_EB
model type: MM_QP
model scaler: 0.0
model input dim: 20
model output dim: 1
model medium bound: EB
timestep: 100000
training set size (10, 20) (10, 1)
QP-Loss 1 0.2272104 0.07015557
QP-Loss 10 0.20700264 0.06390637
QP-Loss 100 0.04306359 0.012898573
QP-Loss 1000 0.0087860245 0.0027652902
QP-Loss 2000 0.0054735364 0.0017401526
QP-Loss 3000 0.004210529 0.0013475176
QP-Loss 4000 0.0034903612 0.0011190932
QP-Loss 5000 0.0030024287 0.00096238527
QP-Loss 6000 0.0026370687 0.00084410864
QP-Loss 7000 0.002346762 0.00074943283
QP-Loss 8000 0.0021079662 0.00067134644
QP-Loss 9000 0.001907666 0.0006060476
QP-Loss 10000 0.001737138 0.0005507308
QP-Loss 11000 0.0015902579 0.00050328433
QP-Loss 12000 0.0014626117 0.00046219255
QP-Loss 13000 0.0013508114 0.000426336
QP-Loss 14000 0.0012524214 0.00039489506
QP-Loss 15000 0.0011654419 0.00036718705
QP-Loss 16000 0.0010882496 0.00034265674
QP-Loss 17000 0.0010195186 0.000320866
QP-Loss 18000 0.00095814106 0.

In [23]:
model.b_int.shape, model.b_ext.shape, model.Pout.shape, model.Pin.shape, model.S.shape, model.V2M.shape, model.model_type

((1000, 67), (154,), (1, 154), (20, 154), (72, 154), (72, 154), 'MM_QP')

In [62]:
model.Pout.shape, np.argmax(model.Pout), model.Pout.flatten()[24], model.get_parameter()

((1, 154), 24, 1.0, None)

In [64]:
parameter = TrainingSet()
parameter.load(model.trainingfile)
parameter.model.reactions[24]

Reaction identifier,BIOMASS_Ecoli_core_w_GAM
Name,Biomass Objective Function with GAM
Memory address,0x1473c3ac748
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP C10H12N5O13P3 + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [41]:
np.matmul(model.X[0], model.Pin).shape

(154,)

In [71]:
model.Pin[15:, :]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.,